<a href="https://colab.research.google.com/github/norman-AI-2025/hackathon-2025/blob/main/Loan_Risk_App_Cloudflare_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import time
import os
import sys
import re
import urllib.request

# ==========================================
# PART 1: SETUP AND LAUNCH APP
# ==========================================

print("--- PART 1: Installing Dependencies & Launching App ---")

# 1. Install Dependencies
print("Installing Python dependencies (this may take a minute)...")
!pip install -q streamlit pandas transformers torch

# 2. Write the Streamlit App File
loan_risk_app_code = '''
import pandas as pd
import streamlit as st
from typing import Callable
import numpy as np
import time

# --- CONSTANTS AND CONFIGURATION ---

PAGES = {
    1: "Applicant Info",
    2: "Guarantor Info",
    3: "Loan Details",
    4: "Results & Text Essay"
}

# Options
GENDER_OPTIONS = ["Male", "Female", "Other"]
MARITAL_OPTIONS = ["Single", "Married", "Divorced", "Widowed"]
EDUCATION_OPTIONS = ["Graduate", "Non-Graduate"]
EMPLOYMENT_OPTIONS = ["Salaried", "Self-Employed", "Business Owner", "Unemployed"]
RELATIONSHIP_OPTIONS = ["Spouse", "Parent", "Sibling", "Friend", "Company"]
PURPOSE_OPTIONS = ["Car Loan", "Education", "Personal", "Renovation", "Business Expansion", "Other"]
COLLATERAL_OPTIONS = ["Real Estate", "Vehicle", "Fixed Deposit", "None"]

# --- Text Analysis Setup ---
@st.cache_resource
def load_sentiment_model():
    try:
        from transformers import pipeline
        return pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    except Exception as e:
        return None

classifier = load_sentiment_model()

def compute_text_score(text: str) -> float:
    if classifier is None or pd.isna(text) or str(text).strip() == "":
        return 50.0
    try:
        result = classifier(str(text))[0]
        score = result['score'] * 100
        return score if result['label'] == 'NEGATIVE' else 100 - score
    except Exception:
        return 50.0

# --- Numeric Model ---
def compute_numeric_risk_scores(data: dict) -> dict:
    # Pre-calculations
    total_applicant_income = data['app_monthly_income'] + data['app_other_monthly_income']
    dti_ratio = (data['app_total_monthly_loan_repayment'] + data['app_other_fixed_monthly_commitments']) / (total_applicant_income + 1e-6)

    credit_limit = data['app_total_credit_limit'] if data['app_total_credit_limit'] > 0 else 1e-6
    credit_utilization = data['app_credit_card_outstanding'] / credit_limit

    collateral_value = data['loan_collateral_value'] if data['loan_collateral_value'] > 0 else 1e-6
    ltv_ratio = data['loan_amount_requested'] / collateral_value if data['loan_is_secured'] else 0.0

    numeric_score = 0
    risk_breakdown = {}

    # 1. DTI (Weight: 25)
    dti_risk = 0.0
    if dti_ratio > 0.43: dti_risk = 100.0
    elif dti_ratio > 0.35: dti_risk = 50 + (dti_ratio - 0.35) / (0.43 - 0.35) * 50
    else: dti_risk = min(dti_ratio / 0.35, 1.0) * 50.0
    numeric_score += dti_risk * 0.25
    risk_breakdown['DTI_Risk'] = dti_risk * 0.25

    # 2. Income (Weight: 15)
    income_risk = max(0, 1 - (total_applicant_income / 150000)) * 100
    numeric_score += income_risk * 0.15
    risk_breakdown['Income_Risk'] = income_risk * 0.15

    # 3. Commitments (Weight: 15)
    commitments_risk = sum([
        10 if data['app_has_mortgage'] else 0,
        15 if data['app_has_car_loan'] else 0,
        15 if data['app_has_personal_loan'] else 0,
    ]) / 40 * 100
    numeric_score += commitments_risk * 0.15
    risk_breakdown['Commitments_Risk'] = commitments_risk * 0.15

    # 4. Utilization (Weight: 25)
    utilization_risk = 0.0
    if credit_utilization > 0.60: utilization_risk = 100.0
    elif credit_utilization > 0.30: utilization_risk = 50 + (credit_utilization - 0.30) / (0.60 - 0.30) * 50
    else: utilization_risk = min(credit_utilization / 0.30, 1.0) * 50.0
    numeric_score += utilization_risk * 0.25
    risk_breakdown['Credit_Utilization_Risk'] = utilization_risk * 0.25

    # 5. Term & LTV (Weight: 20)
    term_risk = min(data['loan_tenure_months'] / 360, 1.0) * 100
    numeric_score += term_risk * 0.10
    risk_breakdown['Loan_Term_Risk'] = term_risk * 0.10

    ltv_risk = 0.0
    if data['loan_is_secured']: ltv_risk = min(ltv_ratio / 0.9, 1.0) * 100
    else: ltv_risk = 90
    numeric_score += ltv_risk * 0.10
    risk_breakdown['LTV_Secured_Risk'] = ltv_risk * 0.10

    # 6. Stability (Weight: 5)
    stability_risk = max(0, 1 - (data['app_years_in_job'] / 10)) * 100
    numeric_score += stability_risk * 0.025
    risk_breakdown['Stability_Risk'] = stability_risk * 0.025

    demographic_risk = (data['app_num_dependents'] / 5 + max(0, 50 - data['app_age']) / 50) * 50
    demographic_risk = min(demographic_risk, 100)
    numeric_score += demographic_risk * 0.025
    risk_breakdown['Demographic_Risk'] = demographic_risk * 0.025

    # Penalties
    if data['app_in_legal_proceedings']:
        numeric_score += 30
        risk_breakdown['Legal_Proceedings_Risk'] = 30
    else: risk_breakdown['Legal_Proceedings_Risk'] = 0

    if data['app_convicted_financial_crime']:
        numeric_score += 50
        risk_breakdown['Financial_Crime_Risk'] = 50
    else: risk_breakdown['Financial_Crime_Risk'] = 0

    # Guarantor Bonus
    guarantor_bonus = 0
    if data['guar_exists'] and data['guar_monthly_income'] > data['app_monthly_income'] * 0.5:
        guarantor_bonus = 10

    data['numeric_score'] = np.clip(numeric_score - guarantor_bonus, 0, 100)
    data['risk_breakdown'] = risk_breakdown
    return data

# --- Fusion Model ---
def compute_fusion_risk(data: dict, text_multiplier: float = 0.20) -> dict:
    text_score = compute_text_score(data['loan_essay_text'])
    data['text_score'] = text_score
    numeric_score = data['numeric_score']

    risk_factor = (text_score - 50.0) / 50.0
    score_multiplier = 1.0 + (risk_factor * text_multiplier)

    data["risk_score"] = np.clip(numeric_score * score_multiplier, 0, 100)
    data["score_multiplier"] = score_multiplier

    score = data["risk_score"]
    if score >= 70: data["risk_category"] = "High Risk"
    elif score >= 30: data["risk_category"] = "Medium Risk"
    else: data["risk_category"] = "Low Risk"
    return data

# --- UI Components ---
def init_session_state():
    if 'current_page' not in st.session_state: st.session_state.current_page = 1
    if 'form_data' not in st.session_state:
        st.session_state.form_data = {
            'app_age': 30, 'app_gender': 'Male', 'app_marital_status': 'Single', 'app_num_dependents': 0,
            'app_education': 'Graduate', 'app_employment_status': 'Salaried', 'app_years_in_job': 5,
            'app_total_work_experience': 10, 'app_monthly_income': 50000, 'app_other_monthly_income': 0,
            'app_has_mortgage': False, 'app_has_car_loan': False, 'app_has_personal_loan': False,
            'app_has_credit_card': False, 'app_num_credit_cards': 1, 'app_total_credit_limit': 100000,
            'app_credit_card_outstanding': 10000, 'app_total_monthly_loan_repayment': 5000,
            'app_other_fixed_monthly_commitments': 0, 'app_in_legal_proceedings': False, 'app_convicted_financial_crime': False,
            'guar_exists': False, 'guar_relationship': 'Spouse', 'guar_age': 30, 'guar_employment_status': 'Salaried',
            'guar_monthly_income': 0, 'guar_other_monthly_income': 0, 'guar_total_monthly_loan_repayment': 0,
            'guar_has_credit_card': False, 'guar_num_credit_cards': 0, 'guar_total_credit_limit': 0, 'guar_credit_card_outstanding': 0,
            'loan_amount_requested': 150000, 'loan_tenure_months': 360, 'loan_purpose': 'Personal',
            'loan_is_secured': False, 'loan_collateral_type': 'None', 'loan_collateral_value': 0,
            'loan_essay_text': "I am seeking this loan to consolidate existing high-interest debt...",
            'numeric_score': 0.0, 'text_score': 0.0, 'risk_score': 0.0, 'risk_category': 'N/A',
            'risk_breakdown': {}, 'score_multiplier': 1.0
        }

def page_applicant_info():
    st.header(f"Page 1: Applicant Info ({PAGES[1]})")
    col1, col2, col3 = st.columns(3)
    st.session_state.form_data['app_age'] = col1.number_input("Age (Years)", min_value=18, max_value=100, value=st.session_state.form_data['app_age'], key="app_age")
    st.session_state.form_data['app_gender'] = col2.selectbox("Gender", options=GENDER_OPTIONS, index=GENDER_OPTIONS.index(st.session_state.form_data['app_gender']), key="app_gender")
    st.session_state.form_data['app_marital_status'] = col3.selectbox("Marital Status", options=MARITAL_OPTIONS, index=MARITAL_OPTIONS.index(st.session_state.form_data['app_marital_status']), key="app_marital_status")

    col4, col5, col6 = st.columns(3)
    st.session_state.form_data['app_num_dependents'] = col4.number_input("Number of Dependents", min_value=0, max_value=10, value=st.session_state.form_data['app_num_dependents'], key="app_num_dependents")
    st.session_state.form_data['app_education'] = col5.selectbox("Education Level", options=EDUCATION_OPTIONS, index=EDUCATION_OPTIONS.index(st.session_state.form_data['app_education']), key="app_education")
    st.session_state.form_data['app_employment_status'] = col6.selectbox("Employment Status", options=EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['app_employment_status']), key="app_employment_status")

    col7, col8 = st.columns(2)
    st.session_state.form_data['app_years_in_job'] = col7.number_input("Years in Current Job", min_value=0, max_value=50, value=st.session_state.form_data['app_years_in_job'], key="app_years_in_job")
    st.session_state.form_data['app_total_work_experience'] = col8.number_input("Total Work Experience (Years)", min_value=0, max_value=50, value=st.session_state.form_data['app_total_work_experience'], key="app_total_work_experience")

    st.subheader("Income & Financial Commitments")
    col9, col10 = st.columns(2)
    st.session_state.form_data['app_monthly_income'] = col9.number_input("Monthly Income ($)", min_value=0, value=st.session_state.form_data['app_monthly_income'], step=1000, key="app_monthly_income")
    st.session_state.form_data['app_other_monthly_income'] = col10.number_input("Other Monthly Income ($)", min_value=0, value=st.session_state.form_data['app_other_monthly_income'], step=500, key="app_other_monthly_income")

    st.markdown("---")
    st.subheader("Existing Commitments")
    col_loan1, col_loan2, col_loan3 = st.columns(3)
    st.session_state.form_data['app_has_mortgage'] = col_loan1.checkbox("Has Mortgage?", value=st.session_state.form_data['app_has_mortgage'], key="app_has_mortgage")
    st.session_state.form_data['app_has_car_loan'] = col_loan2.checkbox("Has Car Loan?", value=st.session_state.form_data['app_has_car_loan'], key="app_has_car_loan")
    st.session_state.form_data['app_has_personal_loan'] = col_loan3.checkbox("Has Personal Loan?", value=st.session_state.form_data['app_has_personal_loan'], key="app_has_personal_loan")
    st.session_state.form_data['app_has_credit_card'] = st.checkbox("Has Credit Card?", value=st.session_state.form_data['app_has_credit_card'], key="app_has_credit_card")

    col_credit1, col_credit2, col_credit3 = st.columns(3)
    st.session_state.form_data['app_num_credit_cards'] = col_credit1.number_input("Number of Credit Cards", min_value=0, value=st.session_state.form_data['app_num_credit_cards'], key="app_num_credit_cards")
    st.session_state.form_data['app_total_credit_limit'] = col_credit2.number_input("Total Credit Limit ($)", min_value=0, value=st.session_state.form_data['app_total_credit_limit'], step=10000, key="app_total_credit_limit")
    st.session_state.form_data['app_credit_card_outstanding'] = col_credit3.number_input("Credit Card Outstanding ($)", min_value=0, value=st.session_state.form_data['app_credit_card_outstanding'], step=1000, key="app_credit_card_outstanding")

    col_repay1, col_repay2 = st.columns(2)
    st.session_state.form_data['app_total_monthly_loan_repayment'] = col_repay1.number_input("Total Monthly Loan Repayment ($)", min_value=0, value=st.session_state.form_data['app_total_monthly_loan_repayment'], step=500, key="app_total_monthly_loan_repayment")
    st.session_state.form_data['app_other_fixed_monthly_commitments'] = col_repay2.number_input("Other Fixed Monthly Commitments ($)", min_value=0, value=st.session_state.form_data['app_other_fixed_monthly_commitments'], step=500, key="app_other_fixed_monthly_commitments")

    st.markdown("---")
    st.subheader("Legal Checks")
    st.session_state.form_data['app_in_legal_proceedings'] = st.checkbox("Involved in legal proceedings?", value=st.session_state.form_data['app_in_legal_proceedings'], key="app_in_legal_proceedings")
    st.session_state.form_data['app_convicted_financial_crime'] = st.checkbox("Convicted of financial crime?", value=st.session_state.form_data['app_convicted_financial_crime'], key="app_convicted_financial_crime")

def page_guarantor_info():
    st.header(f"Page 2: Guarantor Info ({PAGES[2]})")
    st.session_state.form_data['guar_exists'] = st.radio("Do you have a guarantor?", options=[True, False], format_func=lambda x: "Yes" if x else "No", index=0 if st.session_state.form_data['guar_exists'] else 1, key="guar_exists")

    if st.session_state.form_data['guar_exists']:
        st.subheader("Guarantor Details")
        col1, col2, col3 = st.columns(3)
        if st.session_state.form_data['guar_relationship'] not in RELATIONSHIP_OPTIONS:
            st.session_state.form_data['guar_relationship'] = RELATIONSHIP_OPTIONS[0]

        st.session_state.form_data['guar_relationship'] = col1.selectbox("Relationship", options=RELATIONSHIP_OPTIONS, index=RELATIONSHIP_OPTIONS.index(st.session_state.form_data['guar_relationship']), key="guar_relationship")
        st.session_state.form_data['guar_age'] = col2.number_input("Age (Years)", min_value=18, max_value=100, value=st.session_state.form_data['guar_age'], key="guar_age")
        st.session_state.form_data['guar_employment_status'] = col3.selectbox("Employment Status", options=EMPLOYMENT_OPTIONS, index=EMPLOYMENT_OPTIONS.index(st.session_state.form_data['guar_employment_status']), key="guar_employment_status")

        col4, col5 = st.columns(2)
        st.session_state.form_data['guar_monthly_income'] = col4.number_input("Monthly Income ($)", min_value=0, value=st.session_state.form_data['guar_monthly_income'], step=1000, key="guar_monthly_income")
        st.session_state.form_data['guar_other_monthly_income'] = col5.number_input("Other Monthly Income ($)", min_value=0, value=st.session_state.form_data['guar_other_monthly_income'], step=500, key="guar_other_monthly_income")
        st.session_state.form_data['guar_total_monthly_loan_repayment'] = st.number_input("Total Monthly Loan Repayment ($)", min_value=0, value=st.session_state.form_data['guar_total_monthly_loan_repayment'], step=500, key="guar_total_monthly_loan_repayment")
        st.session_state.form_data['guar_has_credit_card'] = st.checkbox("Has Credit Card?", value=st.session_state.form_data['guar_has_credit_card'], key="guar_has_credit_card_check")

        col_credit1, col_credit2, col_credit3 = st.columns(3)
        st.session_state.form_data['guar_num_credit_cards'] = col_credit1.number_input("Number of Credit Cards", min_value=0, value=st.session_state.form_data['guar_num_credit_cards'], key="guar_num_credit_cards")
        st.session_state.form_data['guar_total_credit_limit'] = col_credit2.number_input("Total Credit Limit ($)", min_value=0, value=st.session_state.form_data['guar_total_credit_limit'], step=10000, key="guar_total_credit_limit")
        st.session_state.form_data['guar_credit_card_outstanding'] = col_credit3.number_input("Credit Card Outstanding ($)", min_value=0, value=st.session_state.form_data['guar_credit_card_outstanding'], step=1000, key="guar_credit_card_outstanding")
    else:
        for key in list(st.session_state.form_data.keys()):
            if key.startswith('guar_') and key != 'guar_exists':
                if isinstance(st.session_state.form_data[key], (int, float)): st.session_state.form_data[key] = 0
                elif isinstance(st.session_state.form_data[key], bool): st.session_state.form_data[key] = False
                elif key == 'guar_relationship': st.session_state.form_data[key] = 'None'

def page_loan_details():
    st.header(f"Page 3: Loan Details ({PAGES[3]})")
    col1, col2 = st.columns(2)
    st.session_state.form_data['loan_amount_requested'] = col1.number_input("Loan Amount Requested ($)", min_value=1000, value=st.session_state.form_data['loan_amount_requested'], step=10000, key="loan_amount_requested")
    st.session_state.form_data['loan_tenure_months'] = col2.number_input("Loan Tenure (Months)", min_value=12, max_value=480, value=st.session_state.form_data['loan_tenure_months'], step=12, key="loan_tenure_months")
    st.session_state.form_data['loan_purpose'] = st.selectbox("Purpose", options=PURPOSE_OPTIONS, index=PURPOSE_OPTIONS.index(st.session_state.form_data['loan_purpose']), key="loan_purpose")
    st.session_state.form_data['loan_is_secured'] = st.radio("Is this a Secured Loan?", options=[True, False], format_func=lambda x: "Yes (with Collateral)" if x else "No (Unsecured)", index=0 if st.session_state.form_data['loan_is_secured'] else 1, key="loan_is_secured")

    if st.session_state.form_data['loan_is_secured']:
        st.subheader("Collateral Details")
        col_coll1, col_coll2 = st.columns(2)
        collateral_options_secured = [c for c in COLLATERAL_OPTIONS if c != 'None']
        current_collateral = st.session_state.form_data['loan_collateral_type']
        if current_collateral not in collateral_options_secured: current_collateral = collateral_options_secured[0]
        st.session_state.form_data['loan_collateral_type'] = col_coll1.selectbox("Collateral Type", options=collateral_options_secured, index=collateral_options_secured.index(current_collateral), key="loan_collateral_type")
        st.session_state.form_data['loan_collateral_value'] = col_coll2.number_input("Collateral Value ($)", min_value=1000, value=st.session_state.form_data['loan_collateral_value'], step=10000, key="loan_collateral_value")
    else:
        st.session_state.form_data['loan_collateral_type'] = 'None'
        st.session_state.form_data['loan_collateral_value'] = 0

    st.subheader("Loan Purpose Essay")
    st.session_state.form_data['loan_essay_text'] = st.text_area("Reason for loan:", value=st.session_state.form_data['loan_essay_text'], height=200)

def page_results(text_multiplier):
    st.header(f"Page 4: Risk Assessment ({PAGES[4]})")
    if st.button("Submit & Get Final Risk Score", type="primary"):
        with st.spinner("Calculating..."):
            processed_data = compute_numeric_risk_scores(st.session_state.form_data)
            final_results = compute_fusion_risk(processed_data, text_multiplier=text_multiplier)
            st.session_state.form_data.update(final_results)
            time.sleep(1.0)
        st.success("Analysis Complete!")
        result = st.session_state.form_data
        col_res1, col_res2, col_res3, col_res4 = st.columns(4)
        col_res1.metric("Risk Category", result['risk_category'])
        col_res2.metric("Final Risk Score", f"{result['risk_score']:.2f}")
        col_res3.metric("Numeric Base Score", f"{result['numeric_score']:.2f}")
        col_res4.metric("Text Multiplier", f"x{result['score_multiplier']:.2f}")

        st.markdown("---")
        st.subheader("Breakdown")
        breakdown_df = pd.DataFrame({'Risk Component': list(result['risk_breakdown'].keys()), 'Contribution': [f"{v:.2f}" for v in result['risk_breakdown'].values()]})
        st.dataframe(breakdown_df, hide_index=True, use_container_width=True)

def main_app():
    init_session_state()
    st.set_page_config(page_title="Loan Risk App", layout="wide")
    st.title("🏦 Loan Risk Assessment")
    col_nav1, col_nav2 = st.sidebar.columns(2)
    if st.session_state.current_page > 1:
        if col_nav1.button("← Previous"):
            st.session_state.current_page -= 1
            st.rerun()
    if st.session_state.current_page < len(PAGES):
        if col_nav2.button("Next →"):
            st.session_state.current_page += 1
            st.rerun()
    text_multiplier = st.sidebar.slider("Text Impact", 0.0, 0.5, 0.20, 0.05)
    page_func = {1: page_applicant_info, 2: page_guarantor_info, 3: page_loan_details, 4: lambda: page_results(text_multiplier)}.get(st.session_state.current_page)
    page_func()

if __name__ == "__main__":
    main_app()
'''

with open('streamlit_loan_risk_app.py', 'w') as f:
    f.write(loan_risk_app_code)

print("✅ App file created.")

# 3. Clean up old processes
!pkill cloudflared
!pkill streamlit
print("🧹 Old processes cleaned.")

# 4. Launch Streamlit
PORT = 8501
print(f"\n🚀 Launching Streamlit on port {PORT}...")
subprocess.Popen(f"streamlit run streamlit_loan_risk_app.py --server.port {PORT} --server.enableCORS false --server.enableXsrfProtection false", shell=True, preexec_fn=os.setsid, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

--- PART 1: Installing Dependencies & Launching App ---
Installing Python dependencies (this may take a minute)...
✅ App file created.
🧹 Old processes cleaned.

🚀 Launching Streamlit on port 8501...


<Popen: returncode: None args: 'streamlit run streamlit_loan_risk_app.py --s...>

In [2]:
# ==========================================
# PART 2: TUNNEL SETUP (With Health Check)
# ==========================================

print("\n--- PART 2: Establishing Cloudflare Tunnel ---")

# 5. Health Check: Wait for Streamlit to be Ready
print("⏳ Waiting for Streamlit to start (this ensures the tunnel connects successfully)...")
app_ready = False
for i in range(30): # Wait up to 30 seconds
    try:
        with urllib.request.urlopen(f"http://localhost:{PORT}", timeout=1) as response:
            if response.status == 200:
                print("✅ Streamlit is UP and running!")
                app_ready = True
                break
    except:
        time.sleep(1)
        sys.stdout.write(".")
        sys.stdout.flush()

if not app_ready:
    print("\n❌ Error: Streamlit failed to start in 30 seconds. The tunnel will not be opened.")
    print("Please check if the 'streamlit' installation succeeded in Part 1.")
else:
    # 6. Install Cloudflare
    if not os.path.exists("cloudflared-linux-amd64.deb"):
        print("\n⬇️ Installing cloudflared...")
        !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
        !dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

    # 7. Launch Tunnel
    print("🚇 Opening Cloudflare Tunnel...")
    log_file = open("cloudflared.log", 'w')
    subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"], stdout=log_file, stderr=log_file, preexec_fn=os.setsid)

    time.sleep(3)

    # 8. Get URL
    found_url = False
    for i in range(10):
        if os.path.exists("cloudflared.log"):
            with open("cloudflared.log", 'r') as f:
                content = f.read()
                match = re.search(r'(https:\/\/[a-zA-Z0-9-]+\.trycloudflare\.com)', content)
                if match:
                    print(f"\n🎉 SUCCESS! Your app is live at:\n👉 {match.group(1)}")
                    found_url = True
                    break
        time.sleep(1)

    if not found_url:
        print("\n⚠️ Tunnel started, but URL not found yet. Check logs manually: !cat cloudflared.log")


--- PART 2: Establishing Cloudflare Tunnel ---
⏳ Waiting for Streamlit to start (this ensures the tunnel connects successfully)...
.......✅ Streamlit is UP and running!
🚇 Opening Cloudflare Tunnel...

🎉 SUCCESS! Your app is live at:
👉 https://montgomery-dip-conservative-bless.trycloudflare.com


In [ ]:
from google.colab import drive
drive.mount('/content/drive')